# **INFERENTIAL MODELLING**

## **Introduction**
With inferential modelling, we are aiming to understand how altering our features affect the target variable (price). 

To do this properly, we will need to be especially observant of the assumptions of linear regression.

### **1.1 Import Dependencies**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### **1.2 Import our data**

In [2]:
# We'll call our low_prices X1, y1
low_prices_df = pd.read_csv('drive/MyDrive/DS_Projects/kc_house_project/low_prices_df.csv').drop('Unnamed: 0', axis = 1)
X1 = low_prices_df.drop('price', axis = 1)
y1 = low_prices_df['price']

# We'll call our high_prices X2, y2
high_prices_df = pd.read_csv('drive/MyDrive/DS_Projects/kc_house_project/high_prices_df.csv').drop('Unnamed: 0', axis = 1)
X2 = high_prices_df.drop('price', axis = 1)
y2 = high_prices_df['price']

In [39]:
continuous_features = ['sqft_living','sqft_lot','sqft_above','sqft_basement','lat','long','sqft_living15','sqft_lot15','average_room_size',
                      'floor_area_ratio','bedroom_bathroom_ratio','Seattle_dist_km','Redmond_dist_km','Redmond_Seattle_total_dist','population','land_area','pop_density','water_area','income',
                      'log_bathrooms','log_sqft_living','log_sqft_lot','log_sqft_above','log_sqft_basement','log_lat','log_long','log_sqft_living15','log_sqft_lot15','log_average_room_size',
                      'log_floor_area_ratio','log_bedroom_bathroom_ratio','log_Seattle_dist_km','log_Redmond_dist_km','log_Redmond_Seattle_total_dist','log_population','log_land_area','log_pop_density',
                      'log_water_area','log_income', 'nearby_schools']

categorical_features = ['bedrooms', 'floors','condition', 'grade', 'bathrooms']

## **2.0 Low Prices**
We'll begin with the low prices first i.e. houses up to $1mil.

### **2.1 Train Test Split**

In [10]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1)

Before we make our first model, we should be mindful of multicolinearity.

We'll run a few commands below to get a summary of correlations within our train data.

In [40]:
# First we'll create a dataframe based on the absolute correalation values
corr_df = X_train1[continuous_features].corr().abs().stack().reset_index()

# Level_0 and level_1 refer to the variable names
# We then create a new column that is a tuple of the variable names
corr_df['pairs'] = list(zip(corr_df['level_0'], corr_df['level_1']))

# We'll make the pairs column the index
corr_df.set_index(['pairs'], inplace = True)

# We can then drop the level_0 and level_1 columns
corr_df.drop(columns = ['level_0', 'level_1'], inplace = True)

# We can then rename the '0' column to 'correlation'
corr_df.columns = ['correlation']

# From our heat map we can see that the only perfectly correlation variables are 2 of the same variables
# So we can drop rows that have a correlation of 1
corr_df = corr_df[corr_df['correlation'] != 1]

# Finally, we sort these values by correlation in descending order
corr_df.sort_values(by=['correlation'], ascending = False, inplace = True)

# We also need to get rid of duplicate values e.g. A and B is the same as B and A
corr_df.drop_duplicates(inplace = True)

# Now we can isolate those which are highly correlated
corr_df[corr_df['correlation'] >=0.75]

,correlation
pairs,
"(log_lat, lat)",0.999999
"(log_average_room_size, average_room_size)",0.984847
"(log_sqft_living15, sqft_living15)",0.981270
"(log_population, population)",0.978484
"(log_Redmond_Seattle_total_dist, Redmond_Seattle_total_dist)",0.975728
"(log_bedroom_bathroom_ratio, bedroom_bathroom_ratio)",0.974810
"(sqft_above, log_sqft_above)",0.974121
"(sqft_living, log_sqft_living)",0.972194
"(income, log_income)",0.966419


The results show that these are the most correlated features with each other.

Let's now look at correlations against the target variable.

In [41]:
X_train1_corr = pd.concat([pd.DataFrame(y_train1), X_train1[continuous_features]], axis = 1)
X_train1_corr.corr()['price'].sort_values(ascending = False)

price                             1.000000
log_income                        0.641393
income                            0.615839
sqft_living                       0.576094
log_sqft_living                   0.556004
sqft_living15                     0.516188
log_sqft_living15                 0.505158
sqft_above                        0.470253
log_sqft_above                    0.459928
log_lat                           0.459349
lat                               0.459051
log_bathrooms                     0.395575
log_average_room_size             0.360762
average_room_size                 0.356735
log_floor_area_ratio              0.338506
floor_area_ratio                  0.231827
water_area                        0.220973
log_water_area                    0.199216
pop_density                       0.191538
sqft_basement                     0.182945
log_pop_density                   0.151080
log_bedroom_bathroom_ratio        0.134345
bedroom_bathroom_ratio            0.125641
long       

In this case, we see that income is most correlated with log_price. And that correaltion is in the positive direction.

## **3.0 Low_prices modelling**

### **3.1 Baseline Model**
Our baseline model will be a simple single feature regression model using our most correlate feature. For our analysis, we'll take the logs of both our target and features.

**SUMMARY**
> ***Features:*** Income feature <br>
> ***R sqaured:*** 0.415

In [27]:
import statsmodels.api as sm

In [42]:
low_baseline_model = sm.OLS(np.log(y_train1), sm.add_constant(X_train1['log_income'])).fit()
low_baseline_model.summary()

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.415
Model:                            OLS   Adj. R-squared:                  0.415
Method:                 Least Squares   F-statistic:                     9478.
Date:                Wed, 16 Mar 2022   Prob (F-statistic):               0.00
Time:                        18:35:35   Log-Likelihood:                -4079.7
No. Observations:               13365   AIC:                             8163.
Df Residuals:                   13363   BIC:                             8178.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.9663      0.082     60.606      0.000       4.806       5.127
log_income     0.6996      0.007     97.353      0.000       0.686       0.714
==============================================================================
Omnibus:                      100.128   Durbin-Watson:                   2.008
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              135.744
Skew:                          -0.104   Prob(JB):                     3.34e-30
Kurtosis:                       3.448   Cond. No.                         331.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Our baseline model appears to be performing terribly with a **0.415** R squared value.

### **3.2 Continuous Model**
Our model will now include all continuous variables. We shall also account for multicolinearity.

**SUMMARY**
> ***Features***: Most Correlated continuous features <br>
> ***R sqaured***: 0.627

In [30]:
model_2_features = ['log_income', 'log_sqft_living', 'log_Redmond_dist_km', 'log_average_room_size', 'log_floor_area_ratio']
model_2_df = X_train1[model_2_features]
low_model_2 = sm.OLS(np.log(y_train1), sm.add_constant(model_2_df)).fit()
low_model_2.summary()

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.654
Model:                            OLS   Adj. R-squared:                  0.654
Method:                 Least Squares   F-statistic:                     5045.
Date:                Wed, 16 Mar 2022   Prob (F-statistic):               0.00
Time:                        18:31:06   Log-Likelihood:                -574.12
No. Observations:               13365   AIC:                             1160.
Df Residuals:                   13359   BIC:                             1205.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     5.0732      0.098     51.555      0.000       4.880       5.266
log_income                0.4285      0.007     62.461      0.000       0.415       0.442
log_sqft_living           0.4018      0.008     51.016      0.000       0.386       0.417
log_Redmond_dist_km      -0.1626      0.004    -38.456      0.000      -0.171      -0.154
log_average_room_size     0.0929      0.012      7.908      0.000       0.070       0.116
log_floor_area_ratio      0.1055      0.004     28.049      0.000       0.098       0.113
==============================================================================
Omnibus:                      134.635   Durbin-Watson:                   1.993
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              203.218
Skew:                           0.100   Prob(JB):                     7.44e-45
Kurtosis:                       3.570   Cond. No.                         694.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### **3.3 Continuous + Categorical**
Our model will now include all the prior continuous variables as well as all of our categorical variables.

**SUMMARY**
> ***Features***: Most Correlated continuous features + categoricals<br>
> ***R sqaured***: 0.689

In [89]:
low_model_3_df = X_train1[['log_income', 'log_sqft_living', 'log_Redmond_dist_km', 'log_average_room_size', 'log_floor_area_ratio'] + categorical_features]

low_model_3 = sm.OLS(np.log(y_train1), sm.add_constant(low_model_3_df)).fit()
low_model_3.summary()

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.689
Model:                            OLS   Adj. R-squared:                  0.689
Method:                 Least Squares   F-statistic:                     2957.
Date:                Wed, 16 Mar 2022   Prob (F-statistic):               0.00
Time:                        19:15:11   Log-Likelihood:                 141.31
No. Observations:               13365   AIC:                            -260.6
Df Residuals:                   13354   BIC:                            -178.1
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     5.7385      0.097     58.939      0.000       5.548       5.929
log_income                0.3831      0.007     57.421      0.000       0.370       0.396
log_sqft_living          -0.1891      0.045     -4.246      0.000      -0.276      -0.102
log_Redmond_dist_km      -0.1621      0.004    -40.329      0.000      -0.170      -0.154
log_average_room_size     0.5412      0.045     11.995      0.000       0.453       0.630
log_floor_area_ratio      0.1171      0.004     27.426      0.000       0.109       0.125
bedrooms                  0.1555      0.013     12.052      0.000       0.130       0.181
floors                   -0.0229      0.005     -4.222      0.000      -0.034      -0.012
condition                 0.0782      0.003     23.114      0.000       0.072       0.085
grade                     0.1026      0.003     31.571      0.000       0.096       0.109
bathrooms                 0.1228      0.014      8.481      0.000       0.094       0.151
==============================================================================
Omnibus:                      242.256   Durbin-Watson:                   1.993
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              425.103
Skew:                           0.142   Prob(JB):                     4.90e-93
Kurtosis:                       3.826   Cond. No.                         832.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [128]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

low_model_3_inf = LinearRegression()
low_model_3_inf.fit(low_model_3_df, np.log(y_train1))

low_model_3_test_predictions = low_model_3_inf.predict(X_test1[['log_income', 'log_sqft_living', 'log_Redmond_dist_km', 'log_average_room_size', 'log_floor_area_ratio'] + categorical_features])

low_model_3_test_r2 = r2_score(low_model_3_test_predictions, np.log(y_test1))
low_model_3_test_mse = mean_squared_error(low_model_3_test_predictions, np.log(y_test1), squared = False)
low_model_3_inf.coef_

print(f'test r2: {round(low_model_3_test_r2, 3)} - test rmse: {round(low_model_3_test_mse, 2)}')

test r2: 0.56 - test rmse: 0.24


## **4.0 High Prices**

### **4.1 Train Test Split**

In [115]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2)

Before we make our first model, we should be mindful of multicolinearity.

We'll run a few commands below to get a summary of correlations within our train data.

In [118]:
# First we'll create a dataframe based on the absolute correalation values
corr_df = X_train2[continuous_features].corr().abs().stack().reset_index()

# Level_0 and level_1 refer to the variable names
# We then create a new column that is a tuple of the variable names
corr_df['pairs'] = list(zip(corr_df['level_0'], corr_df['level_1']))

# We'll make the pairs column the index
corr_df.set_index(['pairs'], inplace = True)

# We can then drop the level_0 and level_1 columns
corr_df.drop(columns = ['level_0', 'level_1'], inplace = True)

# We can then rename the '0' column to 'correlation'
corr_df.columns = ['correlation']

# From our heat map we can see that the only perfectly correlation variables are 2 of the same variables
# So we can drop rows that have a correlation of 1
corr_df = corr_df[corr_df['correlation'] != 1]

# Finally, we sort these values by correlation in descending order
corr_df.sort_values(by=['correlation'], ascending = False, inplace = True)

# We also need to get rid of duplicate values e.g. A and B is the same as B and A
corr_df.drop_duplicates(inplace = True)

# Now we can isolate those which are highly correlated
corr_df[corr_df['correlation'] >=0.75]

,correlation
pairs,
"(lat, log_lat)",1.000000
"(Redmond_Seattle_total_dist, log_Redmond_Seattle_total_dist)",0.989018
"(sqft_above, log_sqft_above)",0.988422
"(log_sqft_living, sqft_living)",0.988030
"(log_average_room_size, average_room_size)",0.987529
"(population, log_population)",0.986193
"(income, log_income)",0.985593
"(log_sqft_living15, sqft_living15)",0.983112
"(bedroom_bathroom_ratio, log_bedroom_bathroom_ratio)",0.971769


The results show that these are the most correlated features with each other.

Let's now look at correlations against the target variable.

In [117]:
X_train2_corr = pd.concat([pd.DataFrame(y_train2), X_train2[continuous_features]], axis = 1)
X_train2_corr.corr()['price'].sort_values(ascending = False)

price                             1.000000
sqft_living                       0.336449
log_sqft_living                   0.327971
sqft_living15                     0.291160
log_sqft_living15                 0.280800
log_sqft_above                    0.257688
sqft_above                        0.248596
log_sqft_lot                      0.230454
sqft_lot15                        0.221048
sqft_lot                          0.220828
log_sqft_lot15                    0.220565
log_bathrooms                     0.208266
log_average_room_size             0.171677
average_room_size                 0.150679
log_water_area                    0.115278
income                            0.107483
log_income                        0.096382
log_bedroom_bathroom_ratio        0.092521
bedroom_bathroom_ratio            0.079597
sqft_basement                     0.055582
log_pop_density                   0.048140
water_area                        0.047880
log_lat                           0.036875
lat        

In this case, we see that income is most correlated with sqft_living. And that correaltion is in the positive direction.
However, in this case our correlations with price are significantly lower.

### **4.2 Modelling**

In [125]:
high_model_df = X_train2[['log_income', 'log_sqft_living', 'log_Redmond_dist_km', 'log_average_room_size', 'log_floor_area_ratio']]

high_model = sm.OLS(np.log(y_train2), sm.add_constant(high_model_df)).fit()
high_model.summary()

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.150
Model:                            OLS   Adj. R-squared:                  0.143
Method:                 Least Squares   F-statistic:                     24.01
Date:                Wed, 16 Mar 2022   Prob (F-statistic):           2.74e-22
Time:                        21:17:24   Log-Likelihood:                 99.346
No. Observations:                 688   AIC:                            -186.7
Df Residuals:                     682   BIC:                            -159.5
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    10.6065      0.418     25.345      0.000       9.785      11.428
log_income                0.0450      0.020      2.218      0.027       0.005       0.085
log_sqft_living           0.3768      0.042      8.889      0.000       0.294       0.460
log_Redmond_dist_km      -0.0189      0.020     -0.927      0.355      -0.059       0.021
log_average_room_size    -0.0133      0.047     -0.283      0.777      -0.105       0.079
log_floor_area_ratio     -0.0608      0.016     -3.731      0.000      -0.093      -0.029
==============================================================================
Omnibus:                       64.093   Durbin-Watson:                   2.005
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               79.779
Skew:                           0.805   Prob(JB):                     4.75e-18
Kurtosis:                       3.435   Cond. No.                         844.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [126]:
high_model_inf = LinearRegression()
high_model_inf.fit(high_model_df, np.log(y_train2))

high_model_predictions = high_model_inf.predict(X_test2[['log_income', 'log_sqft_living', 'log_Redmond_dist_km', 'log_average_room_size', 'log_floor_area_ratio']])

high_model_test_r2 = r2_score(high_model_predictions, np.log(y_test2))
high_model_test_mse = mean_squared_error(high_model_predictions, np.log(y_test2), squared = False)
high_model_inf.coef_

print(f'test r2: {round(high_model_test_r2, 3)} - test rmse: {round(high_model_test_mse, 2)}')

test r2: -4.482 - test rmse: 0.22


As we see, the model for our high prices performed terribly. Perhaps we simply need more data.